In [1]:
import pandas as pd
import numpy as np
import pyodbc 

def collect_bp2(sql):
    conn = pyodbc.connect('DSN=BP_DB;UID=bpuser;PWD=kotrabp')   # BPUSER2 에서 데이터 가져옴  // 만약 BPUSER1에서 데이터를 가져올경우 'DSN=BP_DB2'로 변경
    conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le')
    conn.setencoding(encoding='utf-8')
    data= pd.read_sql_query(sql, conn)
    conn.close()
    return data


def collect_bp1(sql):
    conn = pyodbc.connect('DSN=BP_DB2;UID=bpuser;PWD=kotrabp')   # BPUSER2 에서 데이터 가져옴  // 만약 BPUSER1에서 데이터를 가져올경우 'DSN=BP_DB2'로 변경
    conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf-8')
    conn.setdecoding(pyodbc.SQL_WMETADATA, encoding='utf-32le')
    conn.setencoding(encoding='utf-8')
    data= pd.read_sql_query(sql, conn)
    conn.close()
    return data


In [92]:
def make_biz_data():
    sql = ''' SELECT baseyear 기준년도, bizgbncd 사업구분코드, dtlbizname 사업명, biz_from 사업시작일, biz_to 사업종료일,
(SELECT DEPTNAME FROM scrm.TB_CO_DEPT WHERE DEPTCD = A.IMPLDEPTID)  수행부서, procstepname 사업수행단계, seltfname 선정여부명, APPLNO 신청번호, coid 참가KOTRA기업ID, bsno 참가기업사업자번호
FROM SCRM.TB_DA_APPHIST_MAIN A
WHERE appldate > sysdate - 2000;
'''
    data = collect_bp2(sql)
    data = data[~data['사업수행단계'].isin(['사업취소', '취소'])]
    data = data[~data['선정여부명'].isin(['미선정', '불참'])]
    
    biz_cat = pd.DataFrame({'사업구분코드':data['사업구분코드'].value_counts().index.to_list(),
        '사업구분':['기타사업', '수출상담회', '신규수출기업화', '글로벌역량진단', '전시회', '무역사절단',
        '사업파트너연결지원', '유료사업(e)', '지사화/공동물류', '설명회', '월드챔프', '비즈니스세일즈', '열린무역관', '해외공동물류', '프로젝트수주지원']})
    
    data = data.merge(biz_cat, how = 'left')
    data['사업시작일'] = pd.to_datetime(data['사업시작일'])
    data['사업종료일'] = pd.to_datetime(data['사업종료일'].replace({'9999-12-31':np.nan}))
    data['사업시작월'] = data['사업시작일'].apply(lambda x: str(x)[:7].replace('-', ''))
    
    data = data[['기준년도','사업구분코드','사업구분','사업명','사업시작일','사업종료일','수행부서','사업수행단계',
                 '선정여부명','신청번호','참가KOTRA기업ID','참가기업사업자번호']]
    
    return data

In [76]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
biz_data['사업구분코드'].value_counts().index

Index(['OB', 'EF', 'EG', 'GC', 'EX', 'TM', 'RA', 'BR', 'VO', 'PT', 'WC', 'LO',
       'OT', 'BS', 'FP'],
      dtype='object')

In [93]:
biz_data = make_biz_data()
biz_data.head()

,기준년도,사업구분코드,사업구분,사업명,사업시작일,사업종료일,수행부서,사업수행단계,선정여부명,신청번호,참가KOTRA기업ID,참가기업사업자번호
0,2018,PT,설명회,NADCAP HT 2018,2018-07-16,2018-07-19,제조산업팀,신청,참가,PT0200043824,EC115392,5158140973
1,2017,PT,설명회,MDB·글로벌 개발펀드 프로젝트 플라자(설명회_14일 오전),2017-11-14,2017-11-16,인프라 에너지산업팀,신청,참가,PT0200040761,1026547,1018116293
2,2017,PT,설명회,이란 비즈니스 포럼,2017-11-15,2017-11-15,신남방 중동팀,신청,확인,PT0200040351,1083037,2148137726
3,2017,PT,설명회,이란 비즈니스 포럼,2017-11-15,2017-11-15,신남방 중동팀,신청,확인,PT0200040419,EC139233,1198636524
4,2017,PT,설명회,전략시장 진출전략 설명회(대구),2017-12-18,2017-12-18,신남방 중동팀,신청,참가,PT0200040817,EC124347,6212099685


In [94]:
biz_data[['사업구분코드', '사업구분']].drop_duplicates()

,사업구분코드,사업구분
0,PT,설명회
27,EX,전시회
125,EF,수출상담회
466,TM,무역사절단
534,BS,해외공동물류
535,BR,유료사업(e)
576,OB,기타사업
577,RA,사업파트너연결지원
1595,VO,지사화/공동물류
1704,EG,신규수출기업화


In [7]:
sql = '''
SELECT A.BSNO_DECRYPT AS BSNO, D.CONAME, C.COUNTRY_NM AS 국가명, B.EXIMAMTSUM AS 수출금액
FROM BPUSER.TB_CM_EXIMCORP_TOTAL A, BPUSER.TB_CM_EXIMHIST_COUNTRY B, BPUSER.AP_DS_COUNTRY C, BPUSER.TB_CM_CORP D
WHERE A.EXIMCOID = B.EXIMCOID AND A.BSNO_DECRYPT  = D.BSNO AND
B.COUNTRYCD  = C.COUNTRY_CD AND
B.EXPYR = '2022';
'''

df_exp = collect_bp1(sql)


In [95]:
df_exp.head()

,BSNO,CONAME,국가명,수출금액
0,1010166694,부산전구,이탈리아,4547.0
1,1010166694,부산전구,프랑스,10835.0
2,1010166694,부산전구,독일,6243.0
3,1010175441,김장법률사무소,미국,5853.0
4,1010183242,키펙,스페인,26957.0


In [96]:
## 데이터 만들기 :사업자번호	사업시작일	사업종료일	사업구분	사업명	수행부서	2022_수출금액

biz_data_2022 = biz_data[biz_data['기준년도'] == '2022'][['사업시작일', '사업종료일', '사업구분', '사업명', '수행부서', '참가기업사업자번호']]

In [97]:
biz_data_2022 = biz_data_2022.merge(df_exp, how = 'left', 
                    left_on = '참가기업사업자번호', 
                    right_on = 'BSNO' )[['참가기업사업자번호', 'CONAME', '사업시작일', '사업종료일', '사업구분', '사업명', '수행부서', '국가명', '수출금액']]

In [103]:
biz_data_2022.to_csv('BIZ_2022.csv', index = False)

In [100]:
## 함수작성: 무역관명을 입력값으로 받아서 해당사업 참가 기업의 참가사업명, 수출실적을 보여주는 함수

def show_cumstomer_exp(dept):
    df = biz_data_2022[(biz_data_2022['CONAME'].notnull()) & (biz_data_2022['수행부서'].notnull())]
    df.rename(columns = {'참가기업사업자번호':'사업자번호', 'CONAME':'기업명', '국가명':'수출국가명', '수출금액':'수출금액(2022)'}, inplace = True)
    df = df[df['수행부서'].str.contains(dept)].reset_index().drop('index', axis =1)
    return df

In [101]:
show_cumstomer_exp('뉴욕무역관')

,사업자번호,기업명,사업시작일,사업종료일,사업구분,사업명,수행부서,수출국가명,수출금액(2022)
0,2068113719,리베로시스템(주),2022-01-03,NaT,사업파트너연결지원,항목별시장조사,뉴욕무역관,중국,3140.0
1,2068113719,리베로시스템(주),2022-01-03,NaT,사업파트너연결지원,항목별시장조사,뉴욕무역관,미국,46912.0
2,2068113719,리베로시스템(주),2022-01-03,NaT,사업파트너연결지원,항목별시장조사,뉴욕무역관,중국,3140.0
3,2068113719,리베로시스템(주),2022-01-03,NaT,사업파트너연결지원,항목별시장조사,뉴욕무역관,미국,46912.0
4,5518800506,(주)케이비엘러먼트,2022-01-04,NaT,사업파트너연결지원,항목별시장조사,뉴욕무역관,중국,3213.0
...,...,...,...,...,...,...,...,...,...
5881,4468700473,샘표식품(주),2022-09-30,2022-10-02,전시회,2022 뉴욕 O2O 한국 프리미엄 상품전,뉴욕무역관,이탈리아,60080.0
5882,4468700473,샘표식품(주),2022-09-30,2022-10-02,전시회,2022 뉴욕 O2O 한국 프리미엄 상품전,뉴욕무역관,아랍에미리트,81360.0
5883,4468700473,샘표식품(주),2022-09-30,2022-10-02,전시회,2022 뉴욕 O2O 한국 프리미엄 상품전,뉴욕무역관,그리스,15629.0
5884,4468700473,샘표식품(주),2022-09-30,2022-10-02,전시회,2022 뉴욕 O2O 한국 프리미엄 상품전,뉴욕무역관,슬로바키아,60750.0


In [104]:
df = pd.read_csv("BIZ_2022.csv")
df.head()

,참가기업사업자번호,CONAME,사업시작일,사업종료일,사업구분,사업명,수행부서,국가명,수출금액
0,2068113719,리베로시스템(주),2022-01-03,NaN,사업파트너연결지원,항목별시장조사,로스앤젤레스무역관,중국,3140.0
1,2068113719,리베로시스템(주),2022-01-03,NaN,사업파트너연결지원,항목별시장조사,로스앤젤레스무역관,미국,46912.0
2,1248610017,(주)나이스엔테크,2022-01-03,NaN,사업파트너연결지원,항목별시장조사,다낭무역관,코스타리카,5051.0
3,1378633224,NaN,2022-01-04,NaN,사업파트너연결지원,항목별시장조사,하얼빈무역관,NaN,NaN
4,6158173395,주식회사 대길산업,2022-01-04,NaN,사업파트너연결지원,사업파트너 연결지원,토론토무역관,일본,256087.0


In [71]:
kbc_list = biz_data_2022['수행부서'].unique().tolist()

In [74]:
temp_list = []

for kbc in kbc_list:
    try:
        if '무역관' in kbc:
            temp_list.append(kbc)
    except:
        pass

In [75]:
temp_list

['로스앤젤레스무역관',
 '다낭무역관',
 '하얼빈무역관',
 '토론토무역관',
 '모스크바무역관',
 '싱가포르무역관',
 '파리무역관',
 '하노이무역관',
 '뉴욕무역관',
 '바르샤바무역관',
 '나고야무역관',
 '달라스무역관',
 '암스테르담무역관',
 '상파울루무역관',
 '자카르타무역관',
 '보고타무역관',
 '방콕무역관',
 '도하무역관',
 '프놈펜무역관',
 '런던무역관',
 '밀라노무역관',
 '산티아고무역관',
 '마드리드무역관',
 '호치민무역관',
 '브뤼셀무역관',
 '오사카무역관',
 '쿠알라룸푸르무역관',
 '실리콘밸리무역관',
 '홍콩무역관',
 '뉴델리무역관',
 '도쿄무역관',
 '두바이무역관',
 '다롄무역관',
 '코펜하겐무역관',
 '멜버른무역관',
 '요하네스버그무역관',
 '리야드무역관',
 '후쿠오카무역관',
 '양곤무역관',
 '카이로무역관',
 '프랑크푸르트무역관',
 '다카무역관',
 '밴쿠버무역관',
 '마닐라무역관',
 '타이베이무역관',
 '항저우무역관',
 '자그레브무역관',
 '시카고무역관',
 '암다바드무역관',
 '멕시코시티무역관',
 '벵갈루루무역관',
 '바쿠무역관',
 '상하이무역관',
 '무스카트무역관',
 '빈무역관',
 '뭄바이무역관',
 '민스크무역관',
 '부쿠레슈티무역관',
 '키토무역관',
 '부에노스아이레스무역관',
 '헬싱키무역관',
 '콜카타무역관',
 '아테네무역관',
 '암만무역관',
 '디트로이트무역관',
 '키이우무역관',
 '나이로비무역관',
 '비엔티안무역관',
 '아크라무역관',
 '취리히무역관',
 '청두무역관',
 '카라치무역관',
 '워싱턴무역관',
 '함부르크무역관',
 '광저우무역관',
 '베이징무역관',
 '상트페테르부르크무역관',
 '첸나이무역관',
 '테헤란무역관',
 '선양무역관',
 '리마무역관',
 '시드니무역관',
 '우한무역관',
 '마푸투무역관',
 '블라디보스톡무역관',
 '정저우무역관',
 '오클랜드무역관',
 '뮌헨무역관',
